## Exploratory Data Analysis

Understanding and Exploring the dataset and solving the business problem: 
1. Vendor Selection for Profitability
2. Inventory Turnover
3. Product Pricing Optimization

In [1]:
import pandas as pd
import os
import sqlite3

In [2]:
# Creating database connection

conn = sqlite3.connect(r'E:\Vendor Performance Analysis\inventory.db')

In [3]:
# Checking tables that are present inside the database (inventory.db)

tables = pd.read_sql_query("select name from sqlite_master where type = 'table'",conn)
tables

,name
0,purchases
1,purchase_prices
2,sales
3,vendor_invoice
4,vendor_sales_summary


In [4]:
cursor = conn.cursor()

In [6]:
# purchases = pd.read_sql("select * from purchases", conn)
# print(purchases.columns)

# purchase_prices = pd.read_sql("select * from purchase_prices", conn)
# print(purchase_prices.columns)

# sales = pd.read_sql("select * from sales", conn)
# print(sales.columns)

# vendor_invoice = pd.read_sql("select * from vendor_invoice", conn)
# print(vendor_invoice.columns)

In [4]:
# for table in tables['name']:
#     print(f'{table}',':')
#     print('Count of records:', pd.read_sql(f"Select count(*) as count from {table}", conn)['count'].values[0])
#     display(pd.read_sql(f"select * from {table} limit 5", conn))

In [5]:
# purchases = pd.read_sql("select * from purchases", conn)

In [9]:
pd.read_sql("select * from purchases limit 1",conn)

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1


In [10]:
pd.read_sql("select * from purchase_prices limit 1",conn)

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD


In [18]:
pd.read_sql("select * from sales limit 1",conn)

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [12]:
pd.read_sql("select * from vendor_invoice limit 1",conn)

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None


In [5]:
# pd.read_sql("PRAGMA table_info(sales);", conn)

In [26]:
# conn.execute("ALTER TABLE sales RENAME COLUMN VendorNo TO VendorNumber;")
# conn.commit()

### We need to make summary tables for various KPIs

1. Purchase transactions made by vendor
2. Sales transaction data
3. Freight costs for each vendor
4. Actual purchase price from vendors

In [7]:
vendor_summary = pd.read_sql_query('''
select vendornumber, sum(freight) as FreightCost
from Vendor_Invoice
group by vendornumber
''',conn)
vendor_summary

,VendorNumber,FreightCost
0,2,54.16
1,54,0.96
2,60,735.04
3,105,124.78
4,200,12.38
...,...,...
121,98450,1712.04
122,99166,260.18
123,172662,356.68
124,173357,405.00


In [13]:
a= pd.read_sql_query('''
select 
p.vendornumber, 
p.vendorname, p.brand, p.purchaseprice,
pp.volume, 
pp.price as ActualPrice,
sum(p.quantity) as TotalPurchaseQuantity,
sum(p.dollars) as TotalPurchaseDollars
From purchases p
join purchase_prices as pp
on
p.brand = pp.brand
where p.purchaseprice > 0
group by p.vendornumber, p.vendorname, p.brand, p.description, p.purchaseprice, pp.price, pp.volume
order by totalpurchasedollars
''',conn)
print(a)

    VendorNumber                   VendorName  Brand  PurchasePrice Volume  \
0           7245  PROXIMO SPIRITS INC.          3065           0.71     50   
1           3960  DIAGEO NORTH AMERICA INC      6127           1.47    200   
2           3924  HEAVEN HILL DISTILLERIES      9123           0.74     50   
3           8004  SAZERAC CO INC                5683           0.39     50   
4           9815  WINE GROUP INC                8527           1.32    750   
..           ...                          ...    ...            ...    ...   
95          3960  DIAGEO NORTH AMERICA INC      5361           0.79     50   
96          7245  PROXIMO SPIRITS INC.          3152           7.91    750   
97          1650  Circa Wines                  22225           8.05    750   
98         90024  VINILANDIA USA               44917          16.55    750   
99          8004  SAZERAC CO INC                4424           8.39   1750   

    ActualPrice  TotalPurchaseQuantity  TotalPurchaseDollars  


In [15]:
a= pd.read_sql_query('''
select 
p.vendornumber, 
p.vendorname, p.brand, p.purchaseprice,
pp.volume, 
pp.price as ActualPrice,
sum(p.quantity) as TotalPurchaseQuantity,
sum(p.dollars) as TotalPurchaseDollars
From purchases p
join purchase_prices as pp
on
p.brand = pp.brand
where p.purchaseprice > 0
group by p.vendornumber, p.vendorname, p.brand, p.description, p.purchaseprice, pp.price, pp.volume
order by totalpurchasedollars
''',conn)
print(a)

       VendorNumber                   VendorName  Brand  PurchasePrice Volume  \
0              7245  PROXIMO SPIRITS INC.          3065           0.71     50   
1              3960  DIAGEO NORTH AMERICA INC      6127           1.47    200   
2              3924  HEAVEN HILL DISTILLERIES      9123           0.74     50   
3              8004  SAZERAC CO INC                5683           0.39     50   
4              9815  WINE GROUP INC                8527           1.32    750   
...             ...                          ...    ...            ...    ...   
10687          3960  DIAGEO NORTH AMERICA INC      3545          21.89   1750   
10688          3960  DIAGEO NORTH AMERICA INC      4261          16.17   1750   
10689         17035  PERNOD RICARD USA             8068          18.24   1750   
10690          4425        MARTIGNETTI COMPANIES   3405          23.19   1750   
10691          1128  BROWN-FORMAN CORP             1233          26.27   1750   

       ActualPrice  TotalPu

In [17]:
pd.read_sql_query('''
select 
vendorno,
brand,
sum(salesdollars) as TotalSalesDollars,
sum(salesprice) as TotalSalesPrice,
sum(salesquantity) as TotalSalesQuantity,
sum(excisetax) as TotalExciseTax
from sales
group by vendorno, brand
order by totalsalesprice''',conn)

,VendorNo,Brand,TotalSalesDollars,TotalSalesPrice,TotalSalesQuantity,TotalExciseTax
0,8004,5341,5.39,0.49,11,0.58
1,8004,5287,0.98,0.98,2,0.10
2,3252,3933,1.98,0.99,2,0.10
3,3924,9123,1.98,0.99,2,0.10
4,9206,2773,0.99,0.99,1,0.05
...,...,...,...,...,...,...
11267,480,3857,1699669.55,589137.79,32945,60533.43
11268,3960,8680,1884307.34,596574.93,47966,88129.80
11269,17035,2589,2773367.73,614529.34,69627,127931.67
11270,9165,3650,2009508.61,662320.04,41639,32783.09


### After getting 3 summary tables from Purchases, Sales, annd Vendor Invoice, we will:

Merge all these tables into 1 for proper analysis

In [19]:
#Freight Summary CTE
vendor_sales_summary = pd.read_sql_query('''WITH
FreightSummary AS (                         
select 
vendornumber, sum(freight) as FreightCost
from Vendor_Invoice
group by vendornumber
),


PurchaseSummary AS (
select 
p.vendornumber, 
p.vendorname, 
p.brand,
p.description,
p.purchaseprice,
pp.volume, 
pp.price as ActualPrice,
sum(p.quantity) as TotalPurchaseQuantity,
sum(p.dollars) as TotalPurchaseDollars
From purchases p
join purchase_prices pp
on
p.brand = pp.brand
where p.purchaseprice > 0
group by p.vendornumber, p.vendorname, p.brand, p.description, p.purchaseprice, pp.price, pp.volume
),


SalesSummary AS(
select 
vendorno as vendornumber,
brand,
sum(salesdollars) as TotalSalesDollars,
sum(salesprice) as TotalSalesPrice,
sum(salesquantity) as TotalSalesQuantity,
sum(excisetax) as TotalExciseTax
from sales
group by vendornumber, brand
)


Select
ps.vendornumber, 
ps.vendorname, 
ps.brand,
ps.description,
ps.purchaseprice,
ps.ActualPrice,
ps.volume, 
ps.TotalPurchaseQuantity,
ps.TotalPurchaseDollars,
ss.TotalSalesQuantity,
ss.TotalSalesDollars,
ss.TotalSalesPrice,
ss.TotalExciseTax,
fs.FreightCost
FROM purchasesummary ps
left join salessummary ss
on ps.vendornumber = ss.vendornumber
and ps.brand = ss.brand
left join freightSummary fs
on ps.vendornumber = fs.vendornumber
order by ps.totalpurchasedollars desc''',conn)

### Data Cleaning

1. Look for table inconsistencies (missing values)
2. Wrong datatypes
3. Duplicates
4. Unnecessary spaces

In [20]:
vendor_sales_summary

,vendornumber,vendorname,brand,description,purchaseprice,ActualPrice,volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750,145080,3811251.60,142049.0,5101919.51,672819.31,260999.20,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750,164038,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750,187407,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750,201682,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750,138109,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750,2,2.64,5.0,15.95,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50,6,2.34,134.0,65.66,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50,2,1.48,2.0,1.98,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200,1,1.47,72.0,143.28,77.61,15.12,257032.07


In [26]:
 vendor_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vendornumber           10692 non-null  int64  
 1   vendorname             10692 non-null  object 
 2   brand                  10692 non-null  int64  
 3   description            10692 non-null  object 
 4   purchaseprice          10692 non-null  float64
 5   ActualPrice            10692 non-null  float64
 6   volume                 10692 non-null  float64
 7   TotalPurchaseQuantity  10692 non-null  int64  
 8   TotalPurchaseDollars   10692 non-null  float64
 9   TotalSalesQuantity     10514 non-null  float64
 10  TotalSalesDollars      10514 non-null  float64
 11  TotalSalesPrice        10514 non-null  float64
 12  TotalExciseTax         10514 non-null  float64
 13  FreightCost            10692 non-null  float64
dtypes: float64(9), int64(3), object(2)
memory usage: 1.1+ 

In [25]:
vendor_sales_summary.isna().sum()

vendornumber               0
vendorname                 0
brand                      0
description                0
purchaseprice              0
ActualPrice                0
volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
FreightCost                0
dtype: int64

In [21]:
# Fill missing cells with fillna()
# vendor_sales_summary.fillna(0, inplace = True)

# CHANGE DATATYPE FROM OBJECT TO FLOAT(64)
# vendor_sales_summary["volume"] = vendor_sales_summary["volume"].astype(float64)

# Remove unnecessary spaces from the table
# vendor_sales_summary["vendorname"].str.strip()

In [22]:
vendor_sales_summary["volume"] = vendor_sales_summary["volume"].astype('float64')

In [23]:
vendor_sales_summary.fillna(0, inplace = True)

In [24]:
vendor_sales_summary["vendorname"] = vendor_sales_summary["vendorname"].str.strip()

### Making new columns for features for further analysis

In [25]:
vendor_sales_summary["Gross Profit"] = vendor_sales_summary["TotalSalesDollars"] - vendor_sales_summary["TotalPurchaseDollars"]

In [26]:
vendor_sales_summary["Profit Margin"] = vendor_sales_summary["Gross Profit"] / vendor_sales_summary["TotalSalesDollars"] * 100

In [27]:
vendor_sales_summary["StockTurnOver"] = vendor_sales_summary["TotalSalesQuantity"]/vendor_sales_summary["TotalPurchaseQuantity"]

In [28]:
vendor_sales_summary["SalesToPurchaseRatio"] = vendor_sales_summary["TotalSalesDollars"] / vendor_sales_summary["TotalPurchaseDollars"]

### Send the data (vendor_sales_summary) back to sql database for: 
1. Further use by any other person
2. For future refenrence purpose

For this we create an empty table and fetch all the values in it

In [50]:
cursor = conn.cursor()

In [29]:
cursor.execute('''create table vendor_sales_summary(
    vendornumber int,
    vendorname varchar(100),
    brand int,
    description varchar(100),
    purchaseprice decimal(10,2),
    ActualPrice decimal(10,2),
    volume decimal(10,2),
    TotalPurchaseQuantity int,
    TotalPurchaseDollars decimal(10,2),
    TotalSalesQuantity int, 
    TotalSalesDollars decimal(10,2),
    TotalSalesPrice decimal(10,2), 
    TotalExciseTax decimal(10,2),
    FreightCost decimal(10,2), 
    GrossProfit decimal(10,2),
    ProfitMargin decimal(10,2),
    StockTurnOver decimal(10,2),
    SalesToPurchaseRatio decimal(10,2),
    PRIMARY KEY(vendornumber,brand)
    );
    ''')

In [30]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

,vendornumber,vendorname,brand,description,purchaseprice,ActualPrice,volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnOver,SalesToPurchaseRatio


In [59]:
vendor_sales_summary.to_sql('vendor_sales_summary', conn, if_exists = 'replace', index = False)

10692

In [32]:
import sqlite3
import pandas as pd
import logging
from Ingestion_db import ingest_db



logging.basicConfig(
filename = "logs/get_vendor_summary.log",
level = logging.DEBUG,
format = "%(asctime)s-%(levelname)s-%(message)s",
filemode = "a"
)

def create_vendor_summary (conn):
    '''this function will merge all 3 tables in 1 table to get te overall vendor summary and adding new columns in the data'''
    vendor_sales_summary = pd.read_sql_query('''WITH
    FreightSummary AS (                         
    select 
    vendornumber, sum(freight) as FreightCost
    from Vendor_Invoice
    group by vendornumber
    ),


    PurchaseSummary AS (
    select 
    p.vendornumber, 
    p.vendorname, 
    p.brand,
    p.description,
    p.purchaseprice,
    pp.volume, 
    pp.price as ActualPrice,
    sum(p.quantity) as TotalPurchaseQuantity,
    sum(p.dollars) as TotalPurchaseDollars
    From purchases p
    join purchase_prices pp
    on
    p.brand = pp.brand
    where p.purchaseprice > 0
    group by p.vendornumber, p.vendorname, p.brand, p.description, p.purchaseprice, pp.price, pp.volume
    ),


    SalesSummary AS(
    select 
    vendorno as vendornumber,
    brand,
    sum(salesdollars) as TotalSalesDollars,
    sum(salesprice) as TotalSalesPrice,
    sum(salesquantity) as TotalSalesQuantity,
    sum(excisetax) as TotalExciseTax
    from sales
    group by vendornumber, brand
    )


    Select
    ps.vendornumber, 
    ps.vendorname, 
    ps.brand,
    ps.description,
    ps.purchaseprice,
    ps.ActualPrice,
    ps.volume, 
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
    FROM purchasesummary ps
    left join salessummary ss
    on ps.vendornumber = ss.vendornumber
    and ps.brand = ss.brand
    left join freightSummary fs
    on ps.vendornumber = fs.vendornumber
    order by ps.totalpurchasedollars desc''',conn)
    
    return vendor_sales_summary
    
    
def clean_data(df):
    '''This function will clean the data'''
    # Change datatype to float
    df['volume'] = df['volume'].astype('float')
    
    # Fill missing cells with fillna()
    df.fillna(0, inplace = True)
    
    # Remove unnecessary spaces from the columns
    df["vendorname"] = df["vendorname"].str.strip()
    df["description"] = df["description"].str.strip()
    
    
    # Creating new columns for better analysis
    df["Gross Profit"] = df["TotalSalesDollars"] - df["TotalPurchaseDollars"]
    df["Profit Margin"] = df["Gross Profit"] / df["TotalSalesDollars"] * 100
    df["StockTurnOver"] = df["TotalSalesQuantity"]/df["TotalPurchaseQuantity"]
    df["SalesToPurchaseRatio"] = df["TotalSalesDollars"] / df["TotalPurchaseDollars"]
    
    
    return df
    
if __name__ == '__main__':

    
    # Creating database connection
    conn = sqlite3.connect(r'E:\Vendor Performance Analysis\inventory.db')
    
    logging.info("Creating Vendor Summary Table......")
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())
    
    logging.info("Cleaning Data.....")
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())
    
    logging.info("Ingesting Data....")
    ingest_db(clean_df, "vendor_sales_summary", conn)
    logging.info("Completed")

vendor_sales_summary
